<h1 align="center">Jingyang`s Final Assignment</h1>

<p style='text-align: justify;'> Hi, glad to meet you at here. I wrote those words/codes to get a training in Python data science. </p>

## Part 1 Plan and Design  

<p style='text-align: justify;'> 
I want to explore the relationship between the venues trending and their positions. It is quite common to think that those hot venues may be next to each other. But the thing is, how close they are and if it is possible for us to find the possibilities of creating a new "hot group". That`s what I want to do with FourSquare.

My protential audience would be self-employed persons and investors. For self-employers like shop owners, they can make a good use of my result to have a better understand of the trend of business district, they can wisely choose where to do there business. For investors, they can also have a better understanding of the business trends, but from higher perspectives. They can use my results to create new business distince more efficiently, which is essential for them to make more money. 

I will be using the data from FourSquare and some geo-related Python libraries. I plan to use the data from the FourSquare about the trending and position of venues and with the help of Python packages to do visualization and machine learning. 

For example, I can get the check-in number of people of each venue, which would be an important factor that determines whether it is a hot place or not. Then, I will calculate the number limit for determining "hot" or not. I plan to choose the top 20% venues. Then I will get their physical position via Python librarys. Then do the visualization and clustering. 
</p>

## Part 2 Preparation of Work

<p style='text-align: justify;'> 
I will work on the data of Downtown Toronto since I have already done some exploration on Toronto. Base on the geographic data of downtown Toronto, FourSquare API would be applied to get the information of venues nearby. 
</p>

In [1]:
import pandas as pd
import numpy as np

In [19]:
data_table = pd.read_html(r"https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")[0]

In [31]:
data = data_table[1:]
data.columns = data_table.iloc[0]
data = data[data["Borough"] != "Not assigned"].reset_index(drop = True)

In [34]:
data["Neighborhood"] = data["Neighborhood"].str.replace(" /", ",")

In [37]:
data.head()

,Postal code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [35]:
geo_posi = pd.read_csv("Geospatial_Coordinates.csv")

In [39]:
geo_posi.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [46]:
ans_df = pd.merge(data, geo_posi, how = 'left', left_on = "Postal code", right_on = "Postal Code").drop(columns = "Postal Code")
ans_df.columns = ["Postal Code", "Borough", "Neighborhood", "Latitude", "Longitude"]
ans_df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [47]:
ans_df.to_csv("Borough_With_Position.csv")

Now I have the geographic data of Toronto. The \"Downtown Toronto\" would be extracted for further analysis.

## Preparative Data Visualization

<p style='text-align: justify;'> First, I need to have a look at downtown toronto on the map. If it does not have a high density of people, then it would be meaningless for me to do the analysis. The density of people would be represented by the density of postal code since they are always positively related.
</p>

In [2]:
r_posi_df = pd.read_csv("Borough_With_Position.csv")

In [3]:
posi_df = r_posi_df[["Postal Code", "Borough", "Neighborhood", "Latitude", "Longitude"]]
posi_df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [9]:
posi_df.groupby('Borough').count().sort_values(by = ['Postal Code'],
                                               ascending=False).head()

,Postal Code,Neighborhood,Latitude,Longitude
Borough,,,,
North York,24,24,24,24
Downtown Toronto,19,19,19,19
Scarborough,17,17,17,17
Etobicoke,12,12,12,12
Central Toronto,9,9,9,9


<p style='text-align: justify;'> So, it looks like North York and Downtown Toronto have the most postal codes, let`s try to do the visualization.
</p>

In [18]:
densi_Bor_df = posi_df.loc[posi_df['Borough'].isin(['North York',
                                               'Downtown Toronto'])]

densi_Bor_df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [19]:
from geopy.geocoders import Nominatim
import folium # map rendering library

In [20]:
address = "Toronto"

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'
      .format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [32]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough in zip(densi_Bor_df['Latitude'],
                                           densi_Bor_df['Longitude'],
                                           densi_Bor_df['Borough']):
    if borough == 'North York':
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(map_toronto)
    else:
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            color='red',
            fill=True,
            fill_color='#cc315a',
            fill_opacity=0.7,
            parse_html=False).add_to(map_toronto)
    
map_toronto

<p style='text-align: justify;'> From the map, we can see that the reason why North York has many postal codes is it is a big area, not because of high density. The Downtown Toronto has a bigger code density, which is consistent with how we think of "Downtown". So I am going to give a further analysis to the downtown area only. 
</p>

In [35]:
dt_Toronto = densi_Bor_df[densi_Bor_df['Borough'] == 'Downtown Toronto'].reset_index(drop = True)

dt_Toronto.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


In [40]:
address = "Downtown Toronto"

geolocator = Nominatim(user_agent="my_explorer")
dt_location = geolocator.geocode(address)
dt_latitude = dt_location.latitude
dt_longitude = dt_location.longitude
print('The geograpical coordinate of Downtown Toronto are {}, {}.'
      .format(dt_latitude, dt_longitude))

The geograpical coordinate of Downtown Toronto are 43.6541737, -79.38081164513409.


In [46]:
map_Downtown_toronto = folium.Map(location=[latitude, longitude], zoom_start=13)

# add markers to map
for lat, lng in zip(dt_Toronto['Latitude'], dt_Toronto['Longitude']):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Downtown_toronto)
    
map_Downtown_toronto